# Libraries

In [4]:
import numpy as np
import os, cv2, copy
from matplotlib import pyplot as plt

from pcdet.datasets import build_dataloader
from pcdet.utils import common_utils, box_utils
from pcdet.config import cfg, log_config_to_file, cfg_from_list, cfg_from_yaml_file


# Move to the root directory
if not 'workdir' in globals():
    workdir = os.getcwd()
    workdir = "/".join(workdir.split('/')[:-1])
%cd "$workdir"

# Create cache folder
os.makedirs("cache", exist_ok=True)

/home/thuync/Workspace/OpenLidarPerceptron


# Build dataset

In [ ]:
batch_size = 1
workers = 4
dist_train = False
cfg_file = "tools/cfgs/kitti_models/pointpillar_gn.yaml"
log_file = "cache/dataset_debug.txt"

cfg_from_yaml_file(cfg_file, cfg)
logger = common_utils.create_logger(log_file, rank=cfg.LOCAL_RANK)

In [ ]:
train_set, train_loader, train_sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    class_names=cfg.CLASS_NAMES,
    batch_size=batch_size,
    dist=dist_train, workers=workers,
    logger=logger,
    training=True,
)

print("Dataset size:", len(train_set))
print("Classnames:", train_set.class_names)

# Visualize

In [ ]:
idx = 0
data_batch = train_set[idx]
print("Batch components:", data_batch.keys())

points = data_batch['points']
frame_id = data_batch['frame_id']
gt_boxes = data_batch['gt_boxes']
use_lead_xyz = data_batch['use_lead_xyz']
voxels = data_batch['voxels']
voxel_coords = data_batch['voxel_coords']
voxel_num_points = data_batch['voxel_num_points']
image_shape = data_batch['image_shape']

for key, val in data_batch.items():
    if isinstance(val, (str, bool)):
        print("{}: {}".format(key, val))
    else:
        print("{}: {}".format(key, val.shape))

In [ ]:
point_idx = 100
print(points[point_idx])
print(voxel_coords[point_idx])
print(voxels[point_idx])

In [ ]:
xs = np.unique(voxel_coords[:,0])
ys = np.unique(voxel_coords[:,1])
zs = np.unique(voxel_coords[:,2])
print("voxel_coords unique:", len(xs), len(ys), len(zs))

print("gt_boxes:", gt_boxes[3])

# Inspect dataset pipeline

In [2]:
batch_size = 1
workers = 4
dist_train = False
cfg_file = "tools/cfgs/kitti_models/pv_rcnn.yaml"
log_file = "cache/dataset_debug.txt"

cfg_from_yaml_file(cfg_file, cfg)
logger = common_utils.create_logger(log_file, rank=cfg.LOCAL_RANK)

train_set, train_loader, train_sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    class_names=cfg.CLASS_NAMES,
    batch_size=batch_size,
    dist=dist_train, workers=workers,
    logger=logger,
    training=True,
)
print("Dataset size:", len(train_set))
print("Classnames:", train_set.class_names)

/home/thuync/Workspace/OpenLidarPerceptron/pcdet/config.py:73: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  new_config = yaml.load(f)
/home/thuync/Workspace/OpenLidarPerceptron/pcdet/config.py:54: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  yaml_config = yaml.load(f)
2020-07-07 16:23:04,335   INFO  Database filter by min points Car: 14357 => 13532
2020-07-07 16:23:04,336   INFO  Database filter by min points Pedestrian: 2207 => 2168
2020-07-07 16:23:04,337   INFO  Database filter by min points Cyclist: 734 => 705
2020-07-07 16:23:04,362   INFO  Database filter by difficulty Car: 13532 => 10759
2020-07-07 16:23:04,367   INFO  Database filter by difficulty Pedestrian: 2168 => 2075
2020-07-07 16:23:04,369   INFO  Database filter by difficulty Cyclist: 70

Dataset size: 3712
Classnames: ['Car', 'Pedestrian', 'Cyclist']


In [67]:
index = 100
info = copy.deepcopy(train_set.kitti_infos[index])
sample_idx = info['point_cloud']['lidar_idx']

points = train_set.get_lidar(sample_idx)
calib = train_set.get_calib(sample_idx)

img_shape = info['image']['image_shape']
if train_set.dataset_cfg.FOV_POINTS_ONLY:
    pts_rect = calib.lidar_to_rect(points[:, 0:3])
    fov_flag = train_set.get_fov_flag(pts_rect, img_shape, calib)
    points = points[fov_flag]

input_dict = {
    'points': points,
    'frame_id': sample_idx,
    'calib': calib,
}

if 'annos' in info:
    annos = info['annos']
    annos = common_utils.drop_info_with_name(annos, name='DontCare')
    loc, dims, rots = annos['location'], annos['dimensions'], annos['rotation_y']
    gt_names = annos['name']
    gt_boxes_camera = np.concatenate([loc, dims, rots[..., np.newaxis]], axis=1).astype(np.float32)
    gt_boxes_lidar = box_utils.boxes3d_kitti_camera_to_lidar(gt_boxes_camera, calib)

    input_dict.update({
        'gt_names': gt_names,
        'gt_boxes': gt_boxes_lidar
    })
    road_plane = train_set.get_road_plane(sample_idx)
    if road_plane is not None:
        input_dict['road_plane'] = road_plane

data_dict = train_set.prepare_data(data_dict=input_dict)
data_dict['image_shape'] = img_shape

In [72]:
# Examine 'input_dict'

print("info keys: {}\n".format(info.keys()))
print("point_cloud:", info['point_cloud'], '\n')
print("image:", info['image'], '\n')

print('calib:')
for key, val in info['calib'].items():
    print("\t'{}': shape {}".format(key, val.shape))
print()

print('annos:')
for key, val in info['annos'].items():
    print("\t'{}': shape {}".format(key, val.shape))
print()


# Get 'points'
print("points:")
print("\tsample_idx:", sample_idx)
print("\tall points from lidar:", train_set.get_lidar(sample_idx).shape)
print("\tpoints in FOV:", points.shape)
print()

# gt_boxes_lidar
print("gt_boxes_lidar:", gt_boxes_lidar.shape)
print()

# road_plane
print("road_plane:", road_plane.shape)
print()

info keys: dict_keys(['point_cloud', 'image', 'calib', 'annos'])

point_cloud: {'num_features': 4, 'lidar_idx': '000176'} 

image: {'image_idx': '000176', 'image_shape': array([ 375, 1242], dtype=int32)} 

calib:
	'P2': shape (4, 4)
	'R0_rect': shape (4, 4)
	'Tr_velo_to_cam': shape (4, 4)

annos:
	'name': shape (7,)
	'truncated': shape (7,)
	'occluded': shape (7,)
	'alpha': shape (7,)
	'bbox': shape (7, 4)
	'dimensions': shape (7, 3)
	'location': shape (7, 3)
	'rotation_y': shape (7,)
	'score': shape (7,)
	'difficulty': shape (7,)
	'index': shape (7,)
	'gt_boxes_lidar': shape (5, 7)
	'num_points_in_gt': shape (7,)

points:
	sample_idx: 000176
	all points from lidar: (114115, 4)
	points in FOV: (19384, 4)

gt_boxes_lidar: (5, 7)

road_plane: (4,)



In [74]:
# Examine 'data_dict'
print("data_dict:", data_dict.keys())

data_dict: dict_keys(['points', 'frame_id', 'gt_boxes', 'use_lead_xyz', 'voxels', 'voxel_coords', 'voxel_num_points', 'image_shape'])
